In [ ]:
! pip install pygtrie

Разделяем слова по символам и преобразуем их в n-граммы.

In [ ]:
import re

def generate_ngrams(text, max_len):
    text = re.sub(r'[^\w\s]', '', text)  # убираем все специальные символы и пунктуацию из текста
    ngrams = []
    words = text.split(' ')  # делим текст по пробелам
    # Проходим по каждому слову
    for word in words:
        # Проходим по каждому символу в слове
        for symbol in range(len(word)):
            # Создаем n-граммы длиной от 1 до max_len
            for n_symbol in range(1, max_len + 1):
                if symbol + n_symbol <= len(word):  # проверяем, не выходит ли текущая n-грамма за пределы слова
                    ngram = word[symbol:symbol+n_symbol]
                    ngrams.append(ngram)

    return ngrams

Пишем основные функции для построения префиксного дерева, его визуализации и поиска наиболее частотных префиксов

In [ ]:
import pygtrie
from graphviz import Digraph

class Trie:  # реализует префиксное дерево
    def __init__(self):
        self.trie = pygtrie.CharTrie()  # создаем экземпляр CharTrie из библиотеки pygtrie и сохраняем его в атрибуте trie

    def insert(self, ngram):  # метод для добавления n-грамм в дерево
        if ngram in self.trie:
            self.trie[ngram]['frequency'] += 1  # если n-грамма существует в дереве, увеличиваем ее частоту на 1
        else:
            self.trie[ngram] = {'frequency': 1, 'unique_ngrams': {ngram}}  # если n-граммы нет в дереве, добавляем ее с частотой 1 и множеством уникальных n-грамм, содержащим только эту n-грамму
        for pref in range(1, len(ngram)):
            prefix = ngram[:pref]  # проходим по всем префиксам n-граммы
            if prefix in self.trie:  # если префикс существует, добавляем n-грамму в его множество уникальных n-грамм
                self.trie[prefix]['unique_ngrams'].add(ngram)
            else:  # если префикса нет, добавляем его с частотой 0 и множеством уникальных n-грамм, содержащим текущую n-грамму
                self.trie[prefix] = {'frequency': 0, 'unique_ngrams': {ngram}}

    def visualize(self):  #для визуализации префиксного дерева
        dot = Digraph(graph_attr={'splines': 'line'})  # линии прямые
        self._add_nodes(dot, self.trie, "")  # добавляем узлы и ребра в граф
        return dot

    def _add_nodes(self, dot, trie, prefix):  # метод для добавления узлов и ребер в граф
        node_id = f'{prefix}_id'  #  идентификатор узла для текущего префикса
        node_data = trie.get(prefix, {'frequency': 0, 'unique_ngrams': set()})  # получаем данные узла для текущего префикса, или создаем пустой узел, если префикса нет в дереве
        dot.node(node_id, label=f"{prefix[-1:]}\n(freq: {node_data['frequency']}, unique: {len(node_data['unique_ngrams'])})")  # добавляем узел в граф с меткой, содержащей префикс, частоту и количество уникальных n-грамм
        for char in trie.iterkeys(prefix):  # проходим по всем возможным продолжениям текущего префикса
            if char.startswith(prefix) and len(char) == len(prefix) + 1:  # проверяем, является ли char непосредственным продолжением текущего префикса
                child_prefix = char  # устанавливаем child_prefix равным char
                child_id = f'{child_prefix}_id'  # генерируем идентификатор для дочернего узла
                child_data = trie.get(child_prefix, {'frequency': 0, 'unique_ngrams': set()})  # получаем данные дочернего узла
                dot.node(child_id, label=f"{child_prefix[-1]}\n(freq: {child_data['frequency']}, unique: {len(child_data['unique_ngrams'])})")  # добавляем дочерний узел в граф с соответствующей меткой
                dot.edge(node_id, child_id)  # добавляем ребро между текущим узлом и дочерним узлом.
                self._add_nodes(dot, trie, child_prefix)  # рекурсивно вызываем _add_nodes для дочернего узла

    def find_top_prefixes(self, top_n):  # метод для нахождения префиксов с наибольшим соотношением частота/уникальность
        prefix_ratios = []
        for key, value in self.trie.items():
            if value['unique_ngrams']:  # проверяем, содержит ли узел уникальные n-граммы
                ratio = value['frequency'] / len(value['unique_ngrams'])  # вычисляем соотношение частота/уникальность для текущего префикса
                prefix_ratios.append((key, ratio))  # добавляем префикс и его соотношение в список
        prefix_ratios.sort(key=lambda x: x[1], reverse=True)  # сортируем список по соотношению в порядке убывания
        return prefix_ratios[:top_n]  # возвращаем топ префиксов с наибольшим соотношением


def main(text, max_len):
    trie = Trie()
    ngrams = generate_ngrams(text, max_len)

    for ngram in ngrams:
        trie.insert(ngram)

    return trie

Пример использования алгоритма

In [65]:
text = "Золушкины сестры тоже получили приглашение на бал. Они очень обрадовались и сейчас же принялись выбирать наряды и придумывать, как бы причесаться, чтобы удивить всех гостей и понравиться принцу."
max_len = 5  # максимальная длина n-грамм
trie = main(text, max_len)

# Визуализация Trie
dot = trie.visualize()
dot.render('trie_3', format='png', view=True)  # сохраняем файл в формате png

# Нахождение и вывод самых частотных префиксов
top_prefixes = trie.find_top_prefixes(top_n=50)  # ищем 50 самых частотных префиксов
print("Топ префиксы по соотношению частота/уникальность:")
for prefix, ratio in top_prefixes:
    print(f"Префикс: {prefix}, Соотношение: {ratio}")

Топ префиксы по соотношению частота/уникальность:
Префикс: ь, Соотношение: 2.6666666666666665
Префикс: лись, Соотношение: 2.0
Префикс: ись, Соотношение: 2.0
Префикс: сь, Соотношение: 2.0
Префикс: ся, Соотношение: 2.0
Префикс: ться, Соотношение: 2.0
Префикс: же, Соотношение: 2.0
Префикс: бы, Соотношение: 2.0
Префикс: ься, Соотношение: 2.0
Префикс: ть, Соотношение: 1.6666666666666667
Префикс: Золуш, Соотношение: 1.0
Префикс: олушк, Соотношение: 1.0
Префикс: олучи, Соотношение: 1.0
Префикс: оже, Соотношение: 1.0
Префикс: очень, Соотношение: 1.0
Префикс: обрад, Соотношение: 1.0
Префикс: обы, Соотношение: 1.0
Префикс: овали, Соотношение: 1.0
Префикс: остей, Соотношение: 1.0
Префикс: онрав, Соотношение: 1.0
Префикс: лушки, Соотношение: 1.0
Префикс: лучил, Соотношение: 1.0
Префикс: ли, Соотношение: 1.0
Префикс: лис, Соотношение: 1.0
Префикс: лашен, Соотношение: 1.0
Префикс: ушкин, Соотношение: 1.0
Префикс: учили, Соотношение: 1.0
Префикс: умыва, Соотношение: 1.0
Префикс: удиви, Соотношение: 1